## Reexecução

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## Gerando o Dicionário de Descrisções

In [ ]:
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/dic/LIWC2015_pt_desc_only.dic', 'r', encoding='utf-8-sig') as liwc_desc:
    desc_txt = liwc_desc.readlines()

desc_dic = dict()

for line in desc_txt:
    line = line.split('\t')
    
    cat = line[1].replace('\n', '')
    cat = cat.split(' ')[1:]
    aux = ' '
    cat = ''.join(cat)
    
    cat = cat.replace('(', '')
    cat = cat.replace(')', '')
    
    desc_dic[line[0]] = cat
    
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_desc_only.json', 'w', encoding='utf-8') as json_file:
    json.dump(desc_dic, json_file, indent=0)

## Gerado o Dicionário das Palavras do LIWC

In [ ]:
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/dic/LIWC2015_pt_no_desc.dic', 'r', encoding='utf-8-sig') as liwc_words:
    words_txt = liwc_words.readlines()

words_dic = dict()

for line in words_txt:
    tags = []
    line = line.split('\t')
    for element in line:
        if element != '' and element != '\n':
            tags.append(element)
    
    words_dic[tags[0]] = tags[1:]
    
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_no_desc.json', 'w', encoding='utf-8') as json_file:
    json.dump(words_dic, json_file, indent=2)

## Juntado os dois dicionários (criando um dict do LIWC com as descrições ao invés dos números) ?????

In [ ]:
import copy
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_desc_only.json', 'r', encoding='utf-8') as desc_file:
    desc_dict = json.load(desc_file)
#print(desc_dict)    
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_no_desc.json', 'r', encoding='utf-8') as words_file:
    words_dict = json.load(words_file)
#print(words_dict)
for key in words_dict:
    tags = words_dict[key]
    
    for i in range(len(tags)):
        #print(tags[i].replace('/n', ''))
        tags[i] = desc_dict[tags[i].replace('\n', '')]
    
    words_dict[key] = tags

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt.json', 'w', encoding='utf-8') as json_file:
    json.dump(words_dict, json_file, indent=2)

In [22]:
'''
candidatos interessantes:

split 0 - 100
    Alergia
split 0 - 200
    Falta de pontuação prejudica o entendimento do tweet
split 0 - 333-332
    Exemplo simples
split 0 - 838-853
    Vários candidatos numa mesma sentença
'''
candidate = session.query(relation).filter(relation.split == 0).all()[100]

## Aux functions

In [61]:
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text, get_text_splits,
    is_inverted,
)

def get_text(cand):
    return cand.get_parent()._asdict().get('text')

def get_spans(cand):
    return [cand.get_contexts()[0].get_span(), cand.get_contexts()[1].get_span()]

def get_stable_id(cand):
    return cand[0].stable_id.split(':')[0]

def highlight_spans(word_list, span_list):
    s = ''
    for word in word_list:
        if word in span_list:
            s += '*' + word + '* '
        else:
            s += word + ' '
    return s

def print_candidate(cand, switch=False):
    if switch: print("Candidato: " + str(cand), end='\n')
    print("Tweet: " + highlight_spans(get_text(cand).split(' '), get_spans(cand)))

In [66]:
#test stuff
ids = [100,200,332,333,838,839,840,853]
candidates = []
for num in ids:
    candidates.append(session.query(relation).filter(relation.split == 0).all()[num])
#/test stuff

checked_ids = set()

for i in range(len(candidates)):
    stable_id = get_stable_id(candidates[i])
    
    if stable_id not in checked_ids:
        checked_ids.add(stable_id)
        jump = 0
        
        while True:
            if stable_id != get_stable_id(candidates[jump+1]):
                break
            else: jump += 1
        
        print(stable_id + ' ' + str(jump))
        
print(checked_ids)

4509 0
5349 1
4791 0
3549 0
{'4791', '4509', '3549', '5349'}


In [55]:
candidate = [session.query(relation).filter(relation.split == 0).all()[332], session.query(relation).filter(relation.split == 0).all()[332]]

stable_id = candidate[0][0].stable_id.split(':')[0]
#stable_id = 

print(stable_id)
print(type(stable_id))
#print(type(candidate[0].stable_id))
#print(candidate[1].stable_id)
#print(dir(candidate[0].sentence))

4791
<class 'str'>


In [ ]:
%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = [session.query(relation).filter(relation.split == 0).all(), session.query(relation).filter(relation.split == 1).all(), session.query(relation).filter(relation.split == 2).all()]

sw = set(stopwords.words("portuguese"))

quant = dict()

for group in cands:
    for candidate in group:
        for word in get_between_tokens(candidate):
            if word not in sw:
                if word in quant:
                    quant[word] = quant[word] + 1
                else:
                    quant[word] = 1
    

words = []

for key in quant:
    aux = [key,quant[key]]
    words.append(aux)
    
from operator import itemgetter

words = sorted(words, key=itemgetter(1,0))


## NotOLD problema das palavras repetidas

In [ ]:
cands = []
for i in range(838,853):
    cands.append(session.query(relation).filter(relation.split == 0).all()[i])

text = []
for candi in cands:
    text.append(get_text_splits(candi))
#print(len(text)) 15
#print(len(text[1])) 5


print_candidate(cands[0])
for i in range(0, len(text[0])):
    print(i)
    for j in range(0,len(text)):
        print(text[j][i])


## Análise

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dicios = [causas, verbos, causas_e_verbos]

#chegar no candidato e ver se entre os tokens tem algo que a gente quer

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw:
            print(word)
            aux = False
            if causas.get(word):
                causas[word] = causas[word] + 1
                print(word)
                if verbos.get(word):
                    aux = True
                    verbos[word] = verbos[word] + 1
                    causas_e_verbos[word] = causas_e_verbos[word] + 1

            if(aux):
                print(word)
                verbos[word] = verbos[word] + 1

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords


import re

numero = re.compile('\D+')
repeticao = re.compile('\w{2,}')

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dic = {}

#nao repetir frases
#ignorar palavras de 1 letra 

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw and len(word)>1 and numero.match(word):
            if dic.get(word):
                dic[word] = dic[word] + 1
            else:
                dic[word] = 1
                
ordenada = []
maior = ['', 0]

dici = dic


for i in range(0, len(dici)):
    for key in dici:
        if dici[key] > maior[1]:
            maior[0] = key
            maior[1] = dici[key]
    ordenada.append(maior)
    dici.pop(maior[0])
    maior = ['', 0]
                

f = open('/home/lzirondi/Github/snorkel/Scripts/Análise de Palavras/between_tokens_stopFiltered.tsv', 'w', encoding='UTF-8')

for word in ordenada:
    f.write(word[0] + '\t' + str(word[1]) + '\n')
    
f.close()

## Testes

In [25]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print(len(cands))

rep = []

for candidate in cands:
    s = candidate.get_parent().stable_id.split(':')[0]
    
    if s not in rep:
        rep.append(candidate.id)

print(len(rep))
print(len(cands))
#print(dir(cands[0]))
#print(cands[0].get_parent())

print(cands[0].get_parent().stable_id)


Documents: 5984
Sentences: 6031
6398


KeyboardInterrupt: 

In [ ]:
from nltk.corpus import stopwords

sw = set(stopwords.words("portuguese"))

print('com' in sw)